In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from pathlib import Path as P
import matplotlib.pyplot as plt
import plotly.express as px
import pickle
from sklearn.preprocessing import StandardScaler, MinMaxScaler

建模的思路：
1. 小区作为哑巴变量，和时间一起作为自变量；
2. 化学性质是因变量；
3. 用一个两层的MLP模型；

In [2]:
df=pd.read_csv('./问题3-附件14.csv')
df.columns

Index(['year', '放牧强度（intensity）', '放牧小区（plot）', 'SOC土壤有机碳', 'SIC土壤无机碳',
       'STC土壤全碳', '全氮N', '土壤C/N比'],
      dtype='object')

In [3]:
df

,year,放牧强度（intensity）,放牧小区（plot）,SOC土壤有机碳,SIC土壤无机碳,STC土壤全碳,全氮N,土壤C/N比
0,2012,HGI,G13,12.164092,11.550000,23.714092,1.378942,17.394695
1,2012,HGI,G20,13.066348,10.849667,23.916014,1.752764,14.245073
2,2012,HGI,G9,13.474534,11.875000,25.349534,1.648023,17.020659
3,2012,LGI,G12,12.469286,10.491667,22.960953,1.648516,14.458516
4,2012,LGI,G18,12.021940,8.925000,20.946940,1.537228,13.980016
5,2012,LGI,G6,14.327432,13.958333,28.285766,1.895496,15.199783
6,2012,MGI,G11,14.226167,6.712500,20.938667,1.749771,12.481757
7,2012,MGI,G16,11.638731,8.950000,20.588731,1.698386,12.383591
8,2012,MGI,G8,13.677481,9.541667,23.219148,1.763731,13.412295
9,2012,NG,G17,13.973544,10.950000,24.923544,1.659076,16.012928


In [4]:
YEAR,INTENSITY,BLOCK,SOC,SIC,STC,ALLN,CN=['year', '放牧强度（intensity）', '放牧小区（plot）', 'SOC土壤有机碳', 'SIC土壤无机碳',
       'STC土壤全碳', '全氮N', '土壤C/N比']

In [5]:
NUMERIC=[SOC,SIC,STC,ALLN,CN]
scaler=StandardScaler()

scaler.fit(df[NUMERIC])
df[NUMERIC]=scaler.transform(df[NUMERIC])
df_NUMERIC=df[NUMERIC]

df_NUMERIC.to_csv('问题3-因变量.csv', index=False)
pickle.dump(scaler, open('问题3-scaler.pkl', 'wb'))


对因变量，即土壤的化学性质，进行归一化。

In [6]:
df_Year = df[YEAR] - df[YEAR].min()
df_Block=pd.get_dummies(df[BLOCK])

df_INPUT=pd.concat([df_Year,df_Block],axis=1)
df_INPUT.to_csv('问题3-自变量.csv',index=False)

对自变量，即年份和小区，进行归一化和哑巴化（Dummy）。

In [7]:
df_Index=df[[YEAR,INTENSITY,BLOCK]]
df_Index.to_csv('问题3-索引.csv', index=False)

这是展示预测结果用到的索引，其实就是放牧强度和非哑巴化的小区。

In [8]:
scaler.mean_, scaler.scale_

(array([14.92006485,  7.51246398, 22.43252883,  1.82556675, 13.00669993]),
 array([1.7068166 , 2.79228173, 2.43835697, 0.21700343, 2.45867866]))

In [9]:
df_INPUT.shape, df_NUMERIC.shape, df_INPUT.shape

((60, 13), (60, 5), (60, 13))